# 深層学習ノートブック-12 Optimizer
学習ループのコードの保守性を高めるために、パラメータの更新する部分をOptimizerクラスにまとめる。  
* 学習の対象となるparametersおよびlr(学習率)をインスタンスの引数に渡す
* step()メソッドにて各パラメータを更新する処理を記述する
* zero_grad()メソッドにて各パラメータの勾配をゼロにする処理を記述する  

参考：  
* [Udemy講座：「①米国AI開発者がやさしく教える深層学習超入門第一弾【Pythonで実践】」](https://www.udemy.com/course/deeplearning1/learn/lecture/40143418)

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [3]:
class MLP_1(nn.Module):
    def __init__(self, num_in, num_hidden, num_out):
        # 親クラスのinitを呼び出す。
        super().__init__()
        # 全結合層はパラメタを持つのでnn.で定義する
        # num_in, num_hidden, num_outを渡すことにより全結合層を実体化させる
        self.linear_1 = nn.Linear(num_in, num_hidden)
        self.linear_2 = nn.Linear(num_hidden, num_out)

    # nn.Moduleにもforwardメソッドがあり、ここでオーバーライドしている。
    def forward(self, x):
        # デバッグ用
        # z1 = self.linear_1(z)
        # a1 = F.relu(z1)
        # z2 = self.linear_2(a1)
        # 隠れ層の線形変換 →ReLu適用→ 隠れ層の線形変換を一気にやっている。
        z = self.linear_2( F.relu( self.linear_1(x)) )
        return z

In [4]:
# MNISTデータセットの読み込み
dataset = datasets.load_digits()
X = torch.tensor( dataset.data , dtype=torch.float32) 
y = torch.tensor( dataset.target)
# ★ F.cross_entropyの仕様的にはone_hotの形にする必要はない
# y = F.one_hot(y, num_classes=10) 

# hold-out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 標準化
train_mean, train_std = X_train.mean(), X_train.std()
X_train = (X_train - train_mean) / train_std
X_val = (X_val - train_mean) / train_std

# モデルのコンストラクタに渡す引数を定義
num_in = 64
num_hidden = 30
num_out = 10

# モデル定義（２層のMLP）
MLP_model = MLP_1(num_in=num_in, num_hidden=num_hidden, num_out=num_out)
# requires_grad_をTrueに設定。最後に_がつくのは設定するという意味。
MLP_model.requires_grad_(True)

MLP_1(
  (linear_1): Linear(in_features=64, out_features=30, bias=True)
  (linear_2): Linear(in_features=30, out_features=10, bias=True)
)

In [ ]:
# バッチサイズとバッチの個数を定義
batch_size = 32
num_batches = X_train.shape[0] // batch_size + 1
learning_rate = 0.03

# 学習・検証結果格納用辞書
train_results = {}

for i, _ in enumerate(range(100)):
    #　インデックスをシャッフル
    shuffled_indices = np.random.permutation(len(y_train))

    # shuffled_indicesからの取り出し範囲初期化
    idx_start = 0
    idx_end = batch_size

    # 各バッチでの学習データに対するlossを累積する用の変数
    cum_loss = 0

    for _ in range(num_batches):
        # 学習データ定義
        indices_train = shuffled_indices[ idx_start:idx_end ]
        X_train_batch = X_train[indices_train]
        y_train_batch = y_train[indices_train]

        # 順伝播の計算。F.cross_entropyの中でsoftmaxが適用されるので適用不要
        #y_pred = F.softmax(MLP_model(X_train_batch), dim=1)
        y_pred = MLP_model(X_train_batch)

        # 損失計算
        loss = F.cross_entropy(y_pred , y_train_batch)
        cum_loss += loss.item()
        # 逆伝播の計算
        loss.backward()

        # パラメタ更新
        # with torch.no_grad():
        #     MLP_model.linear_1.weight -= learning_rate * MLP_model.linear_1.weight.grad
        #     MLP_model.linear_1.bias -= learning_rate * MLP_model.linear_1.bias.grad
        #     MLP_model.linear_2.weight -= learning_rate * MLP_model.linear_2.weight.grad
        #     MLP_model.linear_2.bias -= learning_rate * MLP_model.linear_2.bias.grad
        # ↓この書き方の方が簡単
        with torch.no_grad():
            for params in MLP_model.parameters():
                params -= learning_rate * params.grad

        # 勾配初期化。nn.Moduleから継承しているので.zero_gradはそのまま使える。
        MLP_model.zero_grad()
        

        # 取り出し範囲更新
        idx_start += batch_size
        idx_end += batch_size
    
    # 検証データに対する損失を計算
    y_pred_val = MLP_model(X_val)
    loss_val = F.cross_entropy(y_pred_val, y_val)

    # 損失、accuracyを記録
    train_results[f"epoch_{i}"] = {
        "Loss_train": cum_loss / num_batches,
        "Loss_val": loss_val.item(),
        "Accuracy": ( (torch.argmax(y_pred_val, dim=1) == y_val).sum() / len(y_val) ).item()
    }

    print(f'epoch_{i}: {train_results[f"epoch_{i}"]}')


epoch_0: {'Loss_train': 2.197762359513177, 'Loss_val': 2.0313637256622314, 'Accuracy': 0.4305555522441864}
epoch_1: {'Loss_train': 1.7720867156982423, 'Loss_val': 1.5172945261001587, 'Accuracy': 0.7055555582046509}
epoch_2: {'Loss_train': 1.237720595465766, 'Loss_val': 1.011123538017273, 'Accuracy': 0.8527777791023254}
epoch_3: {'Loss_train': 0.8398983942137824, 'Loss_val': 0.6964869499206543, 'Accuracy': 0.8833333253860474}
epoch_4: {'Loss_train': 0.6075527045461867, 'Loss_val': 0.5220246911048889, 'Accuracy': 0.8916666507720947}
epoch_5: {'Loss_train': 0.4724813885158963, 'Loss_val': 0.4250306487083435, 'Accuracy': 0.9083333611488342}
epoch_6: {'Loss_train': 0.38948921627468536, 'Loss_val': 0.3576592803001404, 'Accuracy': 0.9166666865348816}
epoch_7: {'Loss_train': 0.3315896646844016, 'Loss_val': 0.30983641743659973, 'Accuracy': 0.9388889074325562}
epoch_8: {'Loss_train': 0.2905773745642768, 'Loss_val': 0.2770136296749115, 'Accuracy': 0.9388889074325562}
epoch_9: {'Loss_train': 0.259